In [1]:
from src.jobs.utils.utility import create_spark_session
spark = create_spark_session("dev_app")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/12 00:43:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# Listar tabelas
spark.sql("SHOW TABLES IN nessie").show()


+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
+---------+---------+-----------+



In [3]:
spark.sql("""CREATE NAMESPACE IF NOT EXISTS nessie.bronze""")

spark.sql("""CREATE NAMESPACE IF NOT EXISTS nessie.silver""")

spark.sql("""CREATE NAMESPACE IF NOT EXISTS nessie.gold""")

DataFrame[]

In [4]:
# Bronze Tables

spark.sql(open("settings/create_bronze_vocation.sql").read())
spark.sql(open("settings/create_bronze_extra.sql").read())
spark.sql(open("settings/create_bronze_skills.sql").read())

# Silver Tables
spark.sql(open("settings/create_silver_vocation.sql").read())
spark.sql(open("settings/create_silver_extra.sql").read())
spark.sql(open("settings/create_silver_skills.sql").read())



SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


DataFrame[]

In [5]:
# Listar tabelas
spark.sql("SHOW TABLES IN nessie").show()


+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|   bronze|    extra|      false|
|   bronze|   skills|      false|
|   bronze| vocation|      false|
|   silver|    extra|      false|
|   silver|   skills|      false|
|   silver| vocation|      false|
+---------+---------+-----------+



In [6]:
spark.stop()

In [1]:
from src.jobs.utils.utility import create_spark_session
spark = create_spark_session("dev_app")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/20 22:58:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
spark.sql("DROP TABLE IF EXISTS nessie.bronze.extra")
spark.sql("DROP TABLE IF EXISTS nessie.silver.extra")

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


DataFrame[]

In [3]:
spark.sql("SHOW TABLES IN nessie").show()


+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|   bronze|   skills|      false|
|   bronze| vocation|      false|
|   silver|   skills|      false|
|   silver| vocation|      false|
+---------+---------+-----------+



In [4]:
spark.stop()

In [2]:
spark.sql(""" CREATE OR REPLACE TABLE nessie.gold.vocation_rank_global
USING iceberg
AS
SELECT
      ROW_NUMBER() OVER (ORDER BY level DESC) AS rank
    , name
    , world
    , vocation
    , level
    , experience
    , world_type
    , current_timestamp() AS updated_at
FROM nessie.silver.vocation
WHERE is_current = true;""")


SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.
26/01/20 22:59:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/20 22:59:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/20 22:59:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/20 23:00:17 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/20 23:00:17 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can caus

DataFrame[]

In [3]:
spark.sql("SHOW TABLES IN nessie").show()

+---------+--------------------+-----------+
|namespace|           tableName|isTemporary|
+---------+--------------------+-----------+
|   bronze|              skills|      false|
|   bronze|            vocation|      false|
|     gold|vocation_rank_global|      false|
|   silver|              skills|      false|
|   silver|            vocation|      false|
+---------+--------------------+-----------+



In [5]:
spark.sql(""" SELECT * FROM nessie.gold.vocation_rank_global """).show()

+----+-----------------+--------+---------------+-----+------------+------------+--------------------+
|rank|             name|   world|       vocation|level|  experience|  world_type|          updated_at|
+----+-----------------+--------+---------------+-----+------------+------------+--------------------+
|   1|        Dejairzin|  inabra|    elder druid| 2997|447914003765|    Open PvP|2026-01-20 22:59:...|
|   2|           Bobeek|    bona|    elder druid| 2958|430572283021|Optional PvP|2026-01-20 22:59:...|
|   3|           Goraca|    bona|master sorcerer| 2955|429183508697|Optional PvP|2026-01-20 22:59:...|
|   4|          Veyllor|gentebra|  royal paladin| 2909|409683078307|Optional PvP|2026-01-20 22:59:...|
|   5|   Warlord Teuzin| belobra|master sorcerer| 2877|396292619074|Optional PvP|2026-01-20 22:59:...|
|   6|            Vepeh| celebra|  royal paladin| 2861|389654187727|Optional PvP|2026-01-20 22:59:...|
|   7|       Mago Morto| celebra|master sorcerer| 2798|364591370274|Optio

In [7]:
spark.stop()